# Deep Research with a Clarifying Agent


In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import requests
import os
from typing import Dict
from IPython.display import display, Markdown
import gradio as gr
from typing import List, Tuple

In [ ]:
load_dotenv(override=True)

### Create an Agent to ask clarifying questions

In [ ]:
CLARIFYING_INSTRUCTIONS = """You are a research analyst. Given a research query, generate 3-5 clarifying 
questions that would help narrow down and improve the research. Focus on:
- Scope and timeframe
- Specific aspects of interest
- Intended use case or audience
- Level of detail needed

Make questions concise and easy to answer."""

class ClarifyingQuestion(BaseModel):
    question: str = Field(description="The clarifying question to ask")
    why_important: str = Field(description="Brief explanation of why this question matters")

class ClarifyingQuestions(BaseModel):
    questions: list[ClarifyingQuestion] = Field(description="List of clarifying questions")

clarifying_agent = Agent  (
    name="ClarifyingAgent",
    instructions=CLARIFYING_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ClarifyingQuestions,  
)


### Create a Search Agent

In [ ]:
SEARCH_INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succinctly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

### Create a Planner Agent

In [ ]:
HOW_MANY_SEARCHES = 3

PLANNER_INSTRUCTIONS = f"""You are a helpful research assistant. Given a query and optional clarifications, 
come up with a set of web searches to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for.
If clarifications are provided, use them to make your searches more targeted and relevant."""

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

### Create a Notification Agent

In [ ]:
@function_tool
def send_research_notification(report_content: str) -> Dict[str, str]:
    """ Send a research report as a Pushover notification """
    url = "https://api.pushover.net/1/messages.json"
    
    import re
    clean_content = re.sub(r'#{1,6}\s*', '', report_content)
    clean_content = re.sub(r'\*\*(.*?)\*\*', r'\1', clean_content)
    clean_content = re.sub(r'\*(.*?)\*', r'\1', clean_content)
    clean_content = re.sub(r'\n\n+', '\n\n', clean_content)
    
    if len(clean_content) > 900:
        clean_content = clean_content[:900] + "...\n\n[Report truncated - see full version in trace]"
    
    data = {
        "token": os.environ.get('PUSHOVER_TOKEN'),
        "user": os.environ.get('PUSHOVER_USER'),
        "message": clean_content,
        "title": "Research Report Complete"
    }
    
    response = requests.post(url, data=data)
    if response.status_code == 200:
        return {"status": "success", "message": "Research report notification sent successfully"}
    else:
        return {"status": "error", "message": f"Error {response.status_code}: {response.text}"}

notification_agent = Agent(
    name="Notification agent",
    instructions="""You are able to send a research report notification via Pushover.
    You will be provided with a detailed markdown report. You should use your tool to send one notification 
    with the report content. The notification should be clean and readable.""",
    tools=[send_research_notification],
    model="gpt-4o-mini",
)

### Create A Writer Agent

In [ ]:
WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, any clarifications provided by the user, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### Core Functions

In [ ]:
async def get_clarifying_questions(query: str):
    """ Use the clarifying agent to generate questions about the query """
    print("Generating clarifying questions...")
    result = await Runner.run(clarifying_agent, f"Research query: {query}")
    print(f"Generated {len(result.final_output.questions)} questions")
    return result.final_output

async def plan_searches(query: str, clarifications: str = ""):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    input_text = f"Query: {query}"
    if clarifications:
        input_text += f"\n\nClarifications from user:\n{clarifications}"
    result = await Runner.run(planner_agent, input_text)
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

async def write_report(query: str, clarifications: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\n\nClarifications: {clarifications}\n\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_notification(report: ReportData):
    """ Use the notification agent to send a notification with the report """
    print("Sending notification...")
    result = await Runner.run(notification_agent, report.markdown_report)
    print("Notification sent")
    return report

### Formatting Functions
def format_questions(questions: ClarifyingQuestions) -> str:
    """ Format clarifying questions for display """
    lines = [
        "Great! I'd like to ask a few clarifying questions to make the research more targeted:\n"
    ]
    
    for i, q in enumerate(questions.questions, 1):
        lines.append(f"**{i}. {q.question}**")
    
    lines.append("\nPlease answer these questions (you can answer all at once or just provide key points).")
    return "\n".join(lines)

### Check if user is answering clarifying questions
def is_answering_questions(history: List[Tuple[str, str]]) -> bool:
    """ Check if user is answering clarifying questions """
    return bool(history and history[-1][1] and "clarifying questions" in history[-1][1].lower())

### Main Research Flow

In [ ]:
async def run_research(query: str, clarifications: str):
    """ Main research flow with clarifications """
    with trace("Research trace"):
        print("Starting research...")
        search_plan = await plan_searches(query, clarifications)
        search_results = await perform_searches(search_plan)
        report = await write_report(query, clarifications, search_results)
        await send_notification(report)  
        print("Hooray!")
        return report.markdown_report

### Gradio UI

In [ ]:
async def chat(message: str, history: List[Tuple[str, str]]):
    """ Handle chat messages through the research flow """
    if is_answering_questions(history):
        # Run research with user's answers
        original_query = history[0][0]
        yield "Perfect! Starting the research now... This may take a minute.\n\n⏳ Planning searches..."
        report = await run_research(original_query, message)
        yield f"✅ Research complete!\n\n{report}\n\n---\n\nFeel free to ask me to research another topic!"
    else:
        # Ask clarifying questions for new topic
        questions = await get_clarifying_questions(message)
        yield format_questions(questions)


ui = gr.ChatInterface(
    fn=chat,
    title="🔍 Deep Research Assistant",
    description="Tell me what you'd like to research, and I'll ask clarifying questions before diving deep!",
    examples=[
        "Latest AI Agent frameworks in 2025",
        "Best practices for RAG systems",
        "State of quantum computing"
    ],
    theme=gr.themes.Soft(primary_hue="blue"),
)

ui.launch(inbrowser=True)